# Regional Inversion Setup

Here we provide a rough template for setting up an inversion, involving gathering an event catalog with moment tensors, collecting observation waveforms and response files, organizing data into the optimal directory structure, and generating ASDFDataSets that can be used in a SeisFlows inversion.


## Event Catalog (Region: Alaska)

Alaska's cool, let's work there. First we'll use ObsPy to gather our initial catalog of events from the past decade in a box bounding Anchorage and Fairbanks.

In [1]:
from obspy import UTCDateTime
from obspy.clients.fdsn import Client

c = Client("USGS")
cat = c.get_events(starttime=UTCDateTime("2010-01-01T00:00:00"), 
                   endtime=UTCDateTime("2020-01-01T00:00:00"), 
                   maxdepth=60.0,
                   minmagnitude=5.0,
                   maxmagnitude=6.0, 
                   minlatitude=59.75, 
                   maxlatitude=65.50, 
                   minlongitude=-154.5, 
                   maxlongitude=-143.789
                  )
cat

15 Event(s) in Catalog:
2019-01-13T16:45:55.437000Z | +61.299, -150.065 | 5.0 ml | manual
2019-01-06T03:45:34.525000Z | +65.407, -153.280 | 5.1 ml | manual
...
2011-06-16T19:06:05.214000Z | +60.765, -151.076 | 5.1 mw | manual
2011-01-23T02:50:04.629000Z | +63.542, -150.865 | 5.2 mw | manual
To see all events call 'print(CatalogObject.__str__(print_all=True))'

Lets have a look at the Event IDs of our events. If we knew these apriori, we could have gathered our catalog based on event ids

In [2]:
from pyatoa.utils.form import format_event_name

event_ids = []
for event in cat:
    event_ids.append(format_event_name(event))
event_ids

['ak019lrs7iu',
 'ak0199za3yf',
 'ak0191pccr7',
 'ak018fe5jk85',
 'ak20421672',
 'ak018fcpk9xi',
 'us1000hyge',
 'ak018fcntv5m',
 'ak018dsf3btv',
 'ak017f7s3c06',
 'ak014dlss56k',
 'ak014b5xf1in',
 'ak013ae2ycca',
 'ak0117oi3hnt',
 'ak011122ukq6']

In [29]:
from obspy import Catalog

events = []
for event_id in event_ids:
    events.append(c.get_events(eventid=event_id)[0])

cat_from_event_ids = Catalog()
cat_from_event_ids.extend(events)
cat_from_event_ids

NameError: name 'cat_from_events_ids' is not defined

## Getting moment tensors

Great, we have an event catalog now, but to do waveform simulations we need moment tensors. Unfortunately it's not straight forward to grab moment tensor information directly from USGS as they do not directly provide XML files. It would be possible to manually generate moment tensor objects from each [individual event pages](https://earthquake.usgs.gov/earthquakes/eventpage/ak019lrs7iu/moment-tensor), but that seems tedious for a tutorial. Instead we'll use good 'ole GCMT. Pyatoa has a function to read .ndk files hosted online by the GCMT team, and find events based on origintime and magnitude.

In [3]:

from pyatoa.core.gatherer import get_gcmt_moment_tensor

events = []
for event in cat:
    origintime = event.preferred_origin().time
    magnitude = event.preferred_magnitude().mag
    try:
        events.append(get_gcmt_moment_tensor(origintime, magnitude))
    except FileNotFoundError:
        print(f"No GCMT event found for: {format_event_name(event)}")
        continue
    
gcmt_catalog = Catalog(events)

No GCMT event found for: ak018fcpk9xi
No GCMT event found for: us1000hyge
No GCMT event found for: ak018fcntv5m
No GCMT event found for: ak013ae2ycca


NameError: name 'Catalog' is not defined

Great, 11 out of 15 isn't bad, we'll go ahead with and use the GCMT catalog that we just collected. However if we wanted to retain the origin information from the USGS catalog, we would need to move the moment tensor objects from the GCMT catalog over to the USGS catalog, an exercise left for the reader...

## Gathering Data

Now we need seismic waveform data for all the events in our catalog

In [4]:
from obspy import Catalog
gcmt_catalog = Catalog(events)


FocalMechanism
	    resource_id: ResourceIdentifier(id="smi:local/ndk/C201901131645A/focal_mechanism")
	   nodal_planes: NodalPlanes(nodal_plane_1=NodalPlane(strike=327.0, dip=44.0, rake=-149.0), nodal_plane_2=NodalPlane(strike=213.0, dip=69.0, rake=-51.0))
	 principal_axes: PrincipalAxes(t_axis=Axis(azimuth=275.0, plunge=15.0, length=3.982e+16), p_axis=Axis(azimuth=167.0, plunge=50.0, length=-3.48e+16), n_axis=Axis(azimuth=17.0, plunge=36.0, length=-5120000000000000.0))
	  moment_tensor: MomentTensor(resource_id=ResourceIdentifier(id="smi:local/ndk/C201901131645A/momenttensor"), derived_origin_id=ResourceIdentifier(id="smi:local/ndk/C201901131645A/origin#cmtorigin"), scalar_moment=3.731e+16, tensor=Tensor(m_rr=-1.96e+16 [uncertainty=1360000000000000.0], m_tt=-1.64e+16 [uncertainty=1340000000000000.0], m_pp=3.59e+16 [uncertainty=890000000000000.0], m_rt=1.53e+16 [uncertainty=780000000000000.0], m_rp=1.43e+16 [uncertainty=750000000000000.0], m_tp=1310000000000000.0 [uncertainty=89000000